In [2]:
import numpy as np
import scipy as sp
import math
import functools as ft
from qiskit.quantum_info import Statevector, SparsePauliOp, Operator

#### Constants

In [3]:
nq = 6

g = math.sqrt(16/3)
m = 1.0

#### Helper Functions

In [5]:
# Define the Pauli Matrices

x = np.matrix([[0.,1.],[1.,0.]])
y = np.matrix([[0.,-1j],[1j,0.]])
z = np.matrix([[1.,0.],[0.,-1.]])
id = sp.sparse.identity(2)

proj0 = (z + id)/2.
proj1 = (id - z)/2.

In [54]:
def ry(angle):
    ang = angle/2
    return np.matrix([[math.cos(ang),-math.sin(ang)],[math.sin(ang),math.cos(ang)]])

In [36]:
# Pad operator on one qubit with identities to expand it to dimension of the full hilbert space
# "op" is the 1 qubit operator. For example Pauli x or Projection

def pad_op(site,total,op):
    return ft.reduce(sp.sparse.kron, [sp.sparse.identity(2**(site)),op,sp.sparse.identity(2**(total-site-1))])

## Construct the Hamiltonian for Qiskit VQE

#### Helper Functions

In [38]:
def pauli_op(op,loc,nq,cons = 1):
    operator = SparsePauliOp.from_sparse_list([(op, [loc], cons)], num_qubits = nq)
    return operator

In [39]:
def projection(which, loc, nq):
    if which == 0:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], 1), ("I", [loc], 1)], num_qubits=nq)/2
    elif which == 1:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], -1), ("I", [loc], 1)], num_qubits=nq)/2
    else:
        raise Exception("Type of projection operator not supported. Choose 0 or 1.")
    return operator

#### Kinetic Hamiltonian

In [40]:
def Hkin(nq):
    op1 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    op2 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op1 += projection(0,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(0,(i+2)%nq,nq))
        op2 += projection(1,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(1,(i+2)%nq,nq))
        
    return (math.sqrt(2)*op1/2 + op2/math.sqrt(2)/2).simplify()

#### Mass Hamiltonian

In [41]:
def Hm(nq,m):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op += projection(0,i,nq).dot(projection(1,(i+1)%nq,nq)) + projection(1,i,nq).dot(projection(0,(i+1)%nq,nq))
        
    return (m*op).simplify()

#### Electric Hamiltonian

In [42]:
def He(nq,g):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    
    for i in range(nq):
        op += projection(1,i,nq) 
        
    return (3*g**2/8*op).simplify()

In [43]:
def He_persite(nq,g,site):
    return 3 * g**2/8 * projection(1,site,nq) 

#### Full Hamiltonian with mass m and gauge coupling g

In [44]:
def Hfull(nq,m,g):
    return (Hkin(nq)+He(nq,g)+Hm(nq,m)).simplify()

# Construct Hamiltonian for QC

In [ ]:
def exp_hkin(nq):
    pass

# Results for comparision

In [ ]:
mathematica_result_6_SU2 = [0.926236987978189, -0.1508356043909894, -0.15083560439098942, \
2.0171952875234895e-24, -0.15083560439098942, 0.02492649322918862, \
-9.16392735034103e-25, -0.0010282134934231357, -0.15083560439098942, \
0.024544483055220404, 0.024926493229188655, -1.0515337362205943e-23, \
1.4320851255807765e-19, -9.482331711350042e-21, \
-0.0010282134934231461, 3.1393305130601976e-22, -0.15083560439098942, \
0.02492649322918864, 0.024544483055220404, -8.572106456264635e-24, \
0.024926493229188648, -0.004181101976514036, 5.557381168626394e-24, \
0.0001743146145043728, -7.582277481027858e-20, 7.124424974409261e-19, \
5.020488570199314e-21, 5.445321829208871e-7, -0.0010282134934231392, \
0.00017431461450437974, -1.6621377479442697e-22, \
-9.773072604810025e-6, -0.15083560439098936, -4.211634526036709e-20, \
0.024926493229188676, -0.0010282134934231461, 0.02454448305522039, \
2.7886691401959976e-21, 3.113863977461585e-24, -9.23256166021131e-23, \
0.024926493229188613, 3.9573168184914087e-19, -0.004181101976514043, \
0.00017431461450436586, -1.3456092921337008e-18, \
5.445321828792538e-7, 0.00017431461450437974, -9.7730726048105e-6, \
1.1640314533079734e-19, -0.0010282134934231496, \
-7.707455516956854e-21, 2.551727720217199e-22, \
-1.0937419234995143e-18, 0.00017431461450434504, \
5.445321828931315e-7, -9.773072604776308e-6, -0.0010282134934233023, \
-1.1102230246251565e-16, 0.00017431461450423402, \
-9.773072604860739e-6, 6.505213034913027e-17, -9.773072604822575e-6, \
-9.773072604811872e-6, 1.6335965486140253e-6]